<a href="https://colab.research.google.com/github/Shakil97/bangla-songs-lyrics-sentiment-analysis/blob/main/SongsSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import tez
import torch
import torch.nn as nn
import transformers
from sklearn import metrics, model_selection
from transformers import AdamW, get_linear_schedule_with_warmup


class BERTDataset:
    def __init__(self, lyrics, target):
        self.lyrics = lyrics
        self.target = target
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "sagorsarker/bangla-bert-base", do_lower_case=True
        )
        self.max_len = 64

    def __len__(self):
        return len(self.lyrics)

    def __getitem__(self, item):
        lyrics = str(self.lyrics[item])
        lyrics = " ".join(lyrics.split())

        inputs = self.tokenizer.encode_plus(
            lyrics,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
        )

        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"]

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "targets": torch.tensor(self.target[item], dtype=torch.float),
        }


class BERTBaseUncased(tez.Model):
    def __init__(self, num_train_steps):
        super().__init__()
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "sagorsarker/bangla-bert-base", do_lower_case=True
        )
        self.bert = transformers.BertModel.from_pretrained(
            "sagorsarker/bangla-bert-base", return_dict=False
        )
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(768, 1)
        self.num_train_steps = num_train_steps
        self.step_scheduler_after = "batch"

    def fetch_optimizer(self):
        param_optimizer = list(self.named_parameters())
        no_decay = ["bias", "LayerNorm.bias"]
        optimizer_parameters = [
            {
                "params": [
                    p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
                ],
                "weight_decay": 0.001,
            },
            {
                "params": [
                    p for n, p in param_optimizer if any(nd in n for nd in no_decay)
                ],
                "weight_decay": 0.0,
            },
        ]
        opt = AdamW(optimizer_parameters, lr=3e-5)
        return opt

    def fetch_scheduler(self):
        sch = get_linear_schedule_with_warmup(
            self.optimizer, num_warmup_steps=0, num_training_steps=self.num_train_steps
        )
        return sch

    def loss(self, outputs, targets):
        if targets is None:
            return None
        return nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 1))

    def monitor_metrics(self, outputs, targets):
        if targets is None:
            return {}
        outputs = torch.sigmoid(outputs).cpu().detach().numpy() >= 0.5
        targets = targets.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, outputs)
        return {"accuracy": accuracy}

    def forward(self, ids, mask, token_type_ids, targets=None):
        _, o_2 = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)
        b_o = self.bert_drop(o_2)
        output = self.out(b_o)
        loss = self.loss(output, targets)
        acc = self.monitor_metrics(output, targets)
        return output, loss, acc


if __name__ == "__main__":
    dfx = pd.read_csv("BanglaSongs.csv").fillna("none")
    dfx.Mood = dfx.Mood.apply(lambda x: 1 if x == "বেদনা" else 0)

    df_train, df_valid = model_selection.train_test_split(
        dfx, test_size=0.2, random_state=42, stratify=dfx.Mood.values
    )

    df_train = df_train.reset_index(drop=True)
    df_valid = df_valid.reset_index(drop=True)

    train_dataset = BERTDataset(
        lyrics=df_train.lyrics.values, target=df_train.Mood.values
    )

    valid_dataset = BERTDataset(
        lyrics=df_valid.lyrics.values, target=df_valid.Mood.values
    )

    n_train_steps = int(len(df_train) / 32 * 10)
    model = BERTBaseUncased(num_train_steps=n_train_steps)

    # model.load("model.bin")
    tb_logger = tez.callbacks.TensorBoardLogger(log_dir=".logs/")
    es = tez.callbacks.EarlyStopping(monitor="valid_loss", model_path="model.bin")
    model.fit(
        train_dataset,
        valid_dataset=valid_dataset,
        train_bs=16,
        device="cuda",
        epochs=100,
        callbacks=[tb_logger, es],
        fp16=True,
    )
    model.save("model.bin")


100%|██████████| 14/14 [00:03<00:00,  4.32it/s, accuracy=0.616, loss=0.634, stage=train]

100%|██████████| 4/4 [00:00<00:00,  4.03it/s, accuracy=0.699, loss=0.597, stage=valid]


Validation score improved (inf --> 0.597308486700058). Saving model!



100%|██████████| 14/14 [00:03<00:00,  4.42it/s, accuracy=0.701, loss=0.53, stage=train]

100%|██████████| 4/4 [00:00<00:00,  4.36it/s, accuracy=0.585, loss=0.627, stage=valid]

  0%|          | 0/14 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5



100%|██████████| 14/14 [00:03<00:00,  4.44it/s, accuracy=0.884, loss=0.37, stage=train]

100%|██████████| 4/4 [00:00<00:00,  4.31it/s, accuracy=0.75, loss=0.616, stage=valid]

  0%|          | 0/14 [00:00<?, ?it/s]

...

EarlyStopping counter: 3 out of 5



100%|██████████| 14/14 [00:03<00:00,  4.42it/s, accuracy=0.991, loss=0.138, stage=train]

100%|██████████| 4/4 [00:00<00:00,  4.26it/s, accuracy=0.699, loss=0.67, stage=valid]

  0%|          | 0/14 [00:00<?, ?it/s]

EarlyStopping counter: 4 out of 5



100%|██████████| 14/14 [00:03<00:00,  4.45it/s, accuracy=0.987, loss=0.139, stage=train]

100%|██████████| 4/4 [00:00<00:00,  4.28it/s, accuracy=0.699, loss=0.67, stage=valid]


EarlyStopping counter: 5 out of 5


In [ ]:
!pip install transformers


     |████████████████████████████████| 1.8MB 8.3MB/s 
     |████████████████████████████████| 890kB 12.1MB/s 
     |████████████████████████████████| 3.2MB 55.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=564ac251c81ab34e79da48346693fa493ca5b5fa59489c054e15565f17ed8063
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!pip install tez
